All comments are above operations that they describe

Starting from all necessary imports

In [ ]:
import sqlite3
import pandas as pd
import datetime as dt
import requests
import csv
import random

Creating/connecting to database

In [ ]:
connection = sqlite3.connect('FlightLeg.db')

Creating a cursor

In [ ]:
cursor = connection.cursor()

Implementation of a method that creates a FlightLeg table

In [ ]:
def create_table(cursor):
  table = ("""CREATE TABLE IF NOT EXISTS FlightLeg
                    (id INTEGER PRIMARY KEY NOT NULL,
                    tailNumber CHAR(10),
                    sourceAirportCode CHAR(3),
                    destinationAirportCode CHAR(3),
                    sourceCountryCode CHAR(3),
                    destinatrionCountryCode CHAR(3),
                    departureTimeUtc DATETIME NOT NULL,
                    landingTimeUtc DATETIME NOT NULL)""")
  
  cursor.execute(table)

Call for create_table method

In [ ]:
create_table(cursor)

This is where the fun begins :D 
Downloading the flightlegs.csv file from provided url

In [ ]:
!wget https://bitpeak.pl/datasets/flightlegs.csv

--2022-01-11 12:09:19--  https://bitpeak.pl/datasets/flightlegs.csv
Resolving bitpeak.pl (bitpeak.pl)... 188.165.23.19
Connecting to bitpeak.pl (bitpeak.pl)|188.165.23.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63132 (62K) [text/csv]
Saving to: ‘flightlegs.csv’

flightlegs.csv      100%[===================>]  61.65K  --.-KB/s    in 0.03s   

2022-01-11 12:09:19 (2.33 MB/s) - ‘flightlegs.csv’ saved [63132/63132]



Opening csv file and converting it to a pandas DataFrame

In [ ]:
data = pd.read_csv('flightlegs.csv', sep=';')
df = pd.DataFrame(data)
df.head()

,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time
0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00
1,A6-ENE,ACC,GHA,ABJ,CIV,2021-11-29 13:35:00,2021-11-29 14:17:00
2,A6-END,DXB,ARE,MNL,PHL,2021-11-27 23:50:11,2021-11-28 07:58:00
3,A6-EPD,IST,TUR,DXB,ARE,2021-11-28 17:51:53,2021-11-28 21:31:00
4,A6-EUU,JED,SAU,DXB,ARE,2021-11-28 08:53:00,2021-11-28 11:10:00


Inserting all data from the dataframe into FlightLeg table

In [ ]:
for row in df.itertuples():
  insert_sql = (f"INSERT INTO FlightLeg (id, tailNumber, sourceAirportCode, destinationAirportCode, sourceCountryCode, destinatrionCountryCode, departureTimeUtc, landingTimeUtc) VALUES ({row[0]}, '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}', '{row[7]}');")
  cursor.execute(insert_sql)

Adding two new columns, flightDuration and flightType

In [ ]:
cursor.execute("ALTER TABLE FlightLeg ADD COLUMN flightDuration INTEGER")
cursor.execute("ALTER TABLE FlightLeg ADD COLUMN flightType TEXT")

Filling flightDuration column using sqlite option

In [ ]:
cursor.execute("SELECT * FROM FlightLeg;")

for row in cursor.fetchall():
  start = dt.datetime.strptime(row[6], '%Y-%m-%d %H:%M:%S')
  end = dt.datetime.strptime(row[7], '%Y-%m-%d %H:%M:%S')
  seconds = end - start
  minutes = int(round((seconds.total_seconds() / 60), 0))

  insert_time = (f"UPDATE FlightLeg SET flightDuration = {minutes} WHERE id = {row[0]};")
  cursor.execute(insert_time)


Filling flightDuration column using dataframe option

In [ ]:
for row in df.itertuples():
  start = dt.datetime.strptime(row[6], '%Y-%m-%d %H:%M:%S')
  end = dt.datetime.strptime(row[7], '%Y-%m-%d %H:%M:%S')
  seconds = end - start
  minutes = int(round((seconds.total_seconds() / 60), 0))

  insert_time = (f"UPDATE FlightLeg SET flightDuration = {minutes} WHERE id = {row[0]};")
  cursor.execute(insert_time)

Filling flightType column using sqlite option

In [ ]:
cursor.execute("SELECT * FROM FlightLeg;")

for row in cursor.fetchall():
  flight_type = ''

  if row[3] == row[5]:
    flight_type = 'D'
  else:
    flight_type = 'I'

  insert_type = (f"UPDATE FlightLeg SET flightType = '{flight_type}' WHERE id = {row[0]};")
  cursor.execute(insert_type)

Filling flightType column using dataframe option

In [ ]:
for row in df.itertuples():
  flight_type = ''

  if row[3] == row[5]:
    flight_type = 'D'
  else:
    flight_type = 'I'

  insert_type = (f"UPDATE FlightLeg SET flightType = '{flight_type}' WHERE id = {row[0]};")
  cursor.execute(insert_type)

Commit of the table, to save all operations to database, from now we will only operate on questions to database

In [ ]:
connection.commit()

Also updating dataframe by adding the same columns and filling them, I will need this later when I will be implementing logic for questions using dataframe option

In [ ]:
flight_time_col = []
for row in df.itertuples():
  start = dt.datetime.strptime(row[6], '%Y-%m-%d %H:%M:%S')
  end = dt.datetime.strptime(row[7], '%Y-%m-%d %H:%M:%S')
  seconds = end - start
  minutes = int(round((seconds.total_seconds() / 60), 0))
  flight_time_col.append(minutes)

df['flight_time'] = flight_time_col

flight_type_col = []
for row in df.itertuples():
  flight_type = ''

  if row[3] == row[5]:
    flight_type = 'D'
  else:
    flight_type = 'I'
  flight_type_col.append(flight_type)

df['flight_type'] = flight_type_col

df.head()

,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time,flight_time,flight_type
0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00,218,I
1,A6-ENE,ACC,GHA,ABJ,CIV,2021-11-29 13:35:00,2021-11-29 14:17:00,42,I
2,A6-END,DXB,ARE,MNL,PHL,2021-11-27 23:50:11,2021-11-28 07:58:00,488,I
3,A6-EPD,IST,TUR,DXB,ARE,2021-11-28 17:51:53,2021-11-28 21:31:00,219,I
4,A6-EUU,JED,SAU,DXB,ARE,2021-11-28 08:53:00,2021-11-28 11:10:00,137,I


Implementation of a logic that prints out tailNumber of a plane with most flights and number of that flights using sqlite option

In [ ]:
cursor.execute("SELECT tailNumber FROM FlightLeg;")
row_list = cursor.fetchall()
mvp_plane = ''
top_count = 0

for i in row_list:
  count = 0
  for j in row_list:
    if j[0] == i[0]:
      count += 1
    else:
      continue
  if count > top_count:
    top_count = count
    mvp_plane = i[0]

if count == 1:
  print('All planes made just one flight! :O')
else:
  print("Plane that made the most flights is "+mvp_plane+" it made "+str(top_count)+" flights!")

Plane that made the most flights is A6-ENE it made 13 flights!


Implementation of a logic that prints out tailNumber of a plane with most flights and number of that flights using dataframe option

In [ ]:
mvp_plane = ''
top_count = 0

for i in df.itertuples():
  count = 0
  for j in df.itertuples():
    if j[1] == i[1]:
      count += 1
    else:
      continue
  if count > top_count:
    top_count = count
    mvp_plane = i[1]

if count == 1:
  print('All planes made just one flight! :O')
else:
  print("Plane that made the most flights is "+mvp_plane+" it made "+str(top_count)+" flights!")

Plane that made the most flights is A6-ENE it made 13 flights!


Implementation of a logic that prints out tailNumber of a plane that made the longest flight and duration of that flight using sqlite option


In [ ]:
cursor.execute("SELECT tailNumber, flightDuration FROM FlightLeg;")
longest_flight = 0
longest_plane = ''

for row in cursor.fetchall():
  if row[1] > longest_flight:
    longest_plane = row[0]
    longest_flight = row[1]
  else:
    continue

print("Plane that made the longest flight is "+longest_plane+" it made "+str(longest_flight)+" minutes flight!")

Plane that made the longest flight is A6-EWH it made 966 minutes flight!


Implementation of a logic that prints out tailNumber of a plane that made the longest flight and duration of that flight using dataframe option

In [ ]:
longest_flight = 0
longest_plane = ''

for row in df.itertuples():
  if int(row[8]) > longest_flight:
    longest_plane = row[1]
    longest_flight = row[8]
  else:
    continue

print("Plane that made the longest flight is "+longest_plane+" it made "+str(longest_flight)+" minutes flight!")

Plane that made the longest flight is A6-EWH it made 966 minutes flight!


Implementation of a logic that prints out tailNumbers of planes that made the longest domestic and international flights and duration of these flights using sqlite option

In [ ]:
cursor.execute("SELECT tailNumber, flightDuration, flightType FROM FlightLeg;")
lngst_d_flight = 0
lngst_i_flight = 0
lngst_d_plane = ''
lngst_i_plane = ''

for row in cursor.fetchall():
  if row[2] == 'D':
    if row[1] > lngst_d_flight:
      lngst_d_plane = row[0]
      lngst_d_flight = row[1]
    else:
      continue
  else:
    if row[1] > lngst_i_flight:
      lngst_i_plane = row[0]
      lngst_i_flight = row[1]
    else:
      continue

print("Plane that made the longest domestic flight is "+lngst_d_plane+" it made "+str(lngst_d_flight)+" minutes flight!")
print("Plane that made the longest international flight is "+lngst_i_plane+" it made "+str(lngst_i_flight)+" minutes flight!")

Plane that made the longest domestic flight is A6-EES it made 126 minutes flight!
Plane that made the longest international flight is A6-EWH it made 966 minutes flight!


Implementation of a logic that prints out tailNumbers of planes that made the shortest domestic and international flights and duration of these flights using sqlite option

In [ ]:
cursor.execute("SELECT tailNumber, flightDuration, flightType FROM FlightLeg;")

shrst_d_flight = lngst_d_flight
shrst_i_flight = lngst_i_flight
shrst_d_plane = ''
shrst_i_plane = ''

for row in cursor.fetchall():
  if row[2] == 'D':
    if row[1] < shrst_d_flight:
      shrst_d_plane = row[0]
      shrst_d_flight = row[1]
    else:
      continue
  else:
    if row[1] < shrst_i_flight:
      shrst_i_plane = row[0]
      shrst_i_flight = row[1]
    else:
      continue

print("Plane that made the shortest domestic flight is "+shrst_d_plane+" it made "+str(shrst_d_flight)+" minutes flight!")
print("Plane that made the shortest international flight is "+shrst_i_plane+" it made "+str(shrst_i_flight)+" minutes flight!")

Plane that made the shortest domestic flight is A6-ECH it made 61 minutes flight!
Plane that made the shortest international flight is A6-ENE it made 42 minutes flight!


Implementation of a logic that prints out tailNumbers of planes that made the longest and the shortest domestic and international flights and duration of these flights using dataframe option

In [ ]:
lngst_d_flight = 0
lngst_i_flight = 0
lngst_d_plane = ''
lngst_i_plane = ''

for row in df.itertuples():
  if row[9] == 'D':
    if row[8] > lngst_d_flight:
      lngst_d_plane = row[1]
      lngst_d_flight = row[8]
    else:
      continue
  else:
    if row[8] > lngst_i_flight:
      lngst_i_plane = row[1]
      lngst_i_flight = row[8]
    else:
      continue

print("Plane that made the longest domestic flight is "+lngst_d_plane+" it made "+str(lngst_d_flight)+" minutes flight!")
print("Plane that made the longest international flight is "+lngst_i_plane+" it made "+str(lngst_i_flight)+" minutes flight!")

shrst_d_flight = lngst_d_flight
shrst_i_flight = lngst_i_flight
shrst_d_plane = ''
shrst_i_plane = ''

for row in df.itertuples():
  if row[9] == 'D':
    if row[8] < shrst_d_flight:
      shrst_d_plane = row[1]
      shrst_d_flight = row[8]
    else:
      continue
  else:
    if row[8] < shrst_i_flight:
      shrst_i_plane = row[1]
      shrst_i_flight = row[8]
    else:
      continue

print("Plane that made the shortest domestic flight is "+shrst_d_plane+" it made "+str(shrst_d_flight)+" minutes flight!")
print("Plane that made the shortest international flight is "+shrst_i_plane+" it made "+str(shrst_i_flight)+" minutes flight!")

Plane that made the longest domestic flight is A6-EES it made 126 minutes flight!
Plane that made the longest international flight is A6-EWH it made 966 minutes flight!
Plane that made the shortest domestic flight is A6-ECH it made 61 minutes flight!
Plane that made the shortest international flight is A6-ENE it made 42 minutes flight!


Implementation of a logic that prints out number of flights that are mistakes (when a plane is doing more than one in the same time), along with tailNumber and departureTimeUtc of that mistaken flight, using sqlite option

In [ ]:
cursor.execute("SELECT tailNumber, departureTimeUtc FROM FlightLeg;")
row_list = cursor.fetchall()
mistakes = []

for i in row_list:
  mstk = 0
  for j in row_list:
    if str(j[1]) == str(i[1]):
      mstk += 1
    else:
      continue
  if mstk > 1:
    mistakes.append(i)

print("There are "+str(len(mistakes))+" records that are mistakes")
print("These records are mistakes:")
for i in mistakes:
  print(i)

There are 69 records that are mistakes
These records are mistakes:
('A6-EVF', '2021-11-30 22:57:00')
('A6-EQI', '2021-11-27 23:12:00')
('A6-ECT', '2021-11-29 14:53:00')
('A6-ENI', '2021-11-28 22:20:00')
('A6-ECQ', '2021-11-29 05:23:00')
('A6-EGZ', '2021-11-29 14:53:00')
('A6-EEU', '2021-11-30 22:29:00')
('A6-ENN', '2021-11-28 05:26:00')
('A6-ENS', '2021-11-30 05:20:00')
('A6-ENJ', '2021-11-29 11:33:00')
('A6-ENW', '2021-11-30 14:09:00')
('A6-EUD', '2021-11-29 13:51:00')
('A6-EVI', '2021-11-29 04:27:00')
('A6-EBU', '2021-11-29 05:07:00')
('A6-EUM', '2021-11-29 04:39:00')
('A6-EOD', '2021-11-28 04:29:00')
('A6-EGZ', '2021-11-30 05:20:00')
('A6-ECF', '2021-11-29 17:59:00')
('A6-ECF', '2021-11-30 16:50:00')
('A6-EVP', '2021-11-29 14:03:00')
('A6-EGE', '2021-11-29 23:33:00')
('A6-ENG', '2021-11-30 05:20:00')
('A6-EEU', '2021-11-29 21:18:00')
('A6-EPS', '2021-11-29 21:18:00')
('A6-ECS', '2021-11-30 15:45:00')
('A6-EOU', '2021-11-29 05:23:00')
('A6-EPM', '2021-11-29 05:07:00')
('A6-EUB', '202

Implementation of a logic that prints out number of flights that are mistakes (when a plane is doing more than one in the same time), along with tailNumber and departureTimeUtc of that mistaken flight, using dataframe option

I also noticed that this option takes much more time to complete than the sqlite option

In [ ]:
mistakes = []

for i in df.itertuples():
  mstk = 0
  for j in df.itertuples():
    if str(j[6]) == str(i[6]):
      mstk += 1
    else:
      continue
  if mstk > 1:
    mistakes.append(i)

print("There are "+str(len(mistakes))+" records that are mistakes")
print("These records are mistakes:")
for i in mistakes:
  print(i[1]+' '+i[6])

There are 69 records that are mistakes
These records are mistakes:
A6-EVF 2021-11-30 22:57:00
A6-EQI 2021-11-27 23:12:00
A6-ECT 2021-11-29 14:53:00
A6-ENI 2021-11-28 22:20:00
A6-ECQ 2021-11-29 05:23:00
A6-EGZ 2021-11-29 14:53:00
A6-EEU 2021-11-30 22:29:00
A6-ENN 2021-11-28 05:26:00
A6-ENS 2021-11-30 05:20:00
A6-ENJ 2021-11-29 11:33:00
A6-ENW 2021-11-30 14:09:00
A6-EUD 2021-11-29 13:51:00
A6-EVI 2021-11-29 04:27:00
A6-EBU 2021-11-29 05:07:00
A6-EUM 2021-11-29 04:39:00
A6-EOD 2021-11-28 04:29:00
A6-EGZ 2021-11-30 05:20:00
A6-ECF 2021-11-29 17:59:00
A6-ECF 2021-11-30 16:50:00
A6-EVP 2021-11-29 14:03:00
A6-EGE 2021-11-29 23:33:00
A6-ENG 2021-11-30 05:20:00
A6-EEU 2021-11-29 21:18:00
A6-EPS 2021-11-29 21:18:00
A6-ECS 2021-11-30 15:45:00
A6-EOU 2021-11-29 05:23:00
A6-EPM 2021-11-29 05:07:00
A6-EUB 2021-11-28 05:26:00
A6-EEN 2021-11-30 14:09:00
A6-EVH 2021-11-30 15:45:00
A6-EOM 2021-11-27 21:08:00
A6-ENS 2021-11-28 04:34:00
A6-EEE 2021-11-29 13:51:00
A6-EQN 2021-11-28 12:56:00
A6-EPX 2021-11-

Implementation of a logic that measures time period between flights of any given plane, I made it so the plane is chosen randomly from the table, so if you run this logic few times you'll see different results, here I used sqlite option

In [ ]:
cursor.execute("SELECT tailNumber, departureTimeUtc, landingTimeUtc FROM FlightLeg;")
row_list = cursor.fetchall()
plane = random.choice(row_list)
flight_list = []
lngst_time_period = []

for i in row_list:
  if i[0] == plane[0]:
    flight_list.append(i)

for j in range(len(flight_list)):
  start = dt.datetime.strptime(flight_list[j][2], '%Y-%m-%d %H:%M:%S')
  if j + 1 < len(flight_list):
    end = dt.datetime.strptime(flight_list[j + 1][1], '%Y-%m-%d %H:%M:%S')
  else:
    pass
  if end > start:
    seconds = end - start
  else:
    seconds = start - end
  minutes = int(round((seconds.total_seconds() / 60), 0))
  lngst_time_period.append(minutes)

for i in range(len(lngst_time_period) - 1):
  for j in range(0, len(lngst_time_period) - i - 1):
    if lngst_time_period[j] < lngst_time_period[j + 1]:
      lngst_time_period[j], lngst_time_period[j + 1] = lngst_time_period[j + 1], lngst_time_period[j]

print("Longest time period between fligths of a plane "+plane[0]+" is "+str(lngst_time_period[0])+" minutes!")
print("Shortest time period between fligths of a plane "+plane[0]+" is "+str(lngst_time_period[-1])+" minutes!")

Longest time period between fligths of a plane A6-EGC is 1895 minutes!
Shortest time period between fligths of a plane A6-EGC is 532 minutes!


Implementation of a logic that measures time period between flights of any given plane, I made it so the plane is chosen randomly from the table, so if you run this logic few times you'll see different results, here I used dataframe option

In [ ]:
row_list = [row for row in df.itertuples()]
plane = random.choice(row_list)
flight_list = []
lngst_time_period = []

for i in row_list:
  if i[1] == plane[1]:
    flight_list.append(i)

for j in range(len(flight_list)):
  start = dt.datetime.strptime(flight_list[j][7], '%Y-%m-%d %H:%M:%S')
  if j + 1 < len(flight_list):
    end = dt.datetime.strptime(flight_list[j + 1][6], '%Y-%m-%d %H:%M:%S')
  else:
    pass
  if end > start:
    seconds = end - start
  else:
    seconds = start - end
  minutes = int(round((seconds.total_seconds() / 60), 0))
  lngst_time_period.append(minutes)

for i in range(len(lngst_time_period) - 1):
  for j in range(0, len(lngst_time_period) - i - 1):
    if lngst_time_period[j] < lngst_time_period[j + 1]:
      lngst_time_period[j], lngst_time_period[j + 1] = lngst_time_period[j + 1], lngst_time_period[j]

print("Longest time period between fligths of a plane "+plane[1]+" is "+str(lngst_time_period[0])+" minutes!")
print("Shortest time period between fligths of a plane "+plane[1]+" is "+str(lngst_time_period[-1])+" minutes!")

Longest time period between fligths of a plane A6-EUN is 3871 minutes!
Shortest time period between fligths of a plane A6-EUN is 459 minutes!


Final commit of the table for safety measures

In [ ]:
connection.commit()

Here is just a print of the table transormed into dataframe again, I used it to check if everything is going right

In [ ]:
cursor.execute("SELECT * FROM FlightLeg")
results = []
for row in cursor.fetchall():
  results.append(row)
print_res = pd.DataFrame(results)
print(print_res)

       0       1    2    3  ...                    6                    7    8  9
0      0  A6-EUC  DXB  ARE  ...  2021-11-30 04:47:00  2021-11-30 08:25:00  218  I
1      1  A6-ENE  ACC  GHA  ...  2021-11-29 13:35:00  2021-11-29 14:17:00   42  I
2      2  A6-END  DXB  ARE  ...  2021-11-27 23:50:11  2021-11-28 07:58:00  488  I
3      3  A6-EPD  IST  TUR  ...  2021-11-28 17:51:53  2021-11-28 21:31:00  219  I
4      4  A6-EUU  JED  SAU  ...  2021-11-28 08:53:00  2021-11-28 11:10:00  137  I
..   ...     ...  ...  ...  ...                  ...                  ...  ... ..
995  995  A6-EOV  DXB  ARE  ...  2021-11-28 13:01:00  2021-11-28 17:48:00  287  I
996  996  A6-EGW  ZAZ  ESP  ...  2021-11-28 17:37:30  2021-11-29 00:08:00  390  I
997  997  A6-EGT  BKK  THA  ...  2021-11-28 13:22:06  2021-11-28 19:33:00  371  I
998  998  A6-EGP  DXB  ARE  ...  2021-11-29 04:51:00  2021-11-29 19:07:00  856  I
999  999  A6-EVJ  LHR  GBR  ...  2021-11-29 16:27:00  2021-11-29 22:29:00  362  I

[1000 rows x 10

From looking at booth options I used, the dataframe and sqlite, I conlude that using dataframes requires less code, but unfortunately sometimes it takes more time, using sqlite we work on the original table, and do not have to transfer it from dataframe to a table first and because of that I think that working more with sqlite is better